# Reducing hospital readmissions

<img src="https://www.bluetooth.com/wp-content/uploads/headers/1903_Header_Hospital.jpg">

## 📖 Background
You work for a consulting company helping a hospital group better understand patient readmissions. The hospital gave you access to ten years of information on patients readmitted to the hospital after being discharged. The doctors want you to assess if initial diagnoses, number of procedures, or other variables could help them better understand the probability of readmission. 

They want to focus follow-up calls and attention on those patients with a higher probability of readmission.

## 💾 The data
You have access to ten years of patient information ([source](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008)):

#### Information in the file
- "age" - age bracket of the patient
- "time_in_hospital" - days (from 1 to 14)
- "n_procedures" - number of procedures performed during the hospital stay
- "n_lab_procedures" - number of laboratory procedures performed during the hospital stay
- "n_medications" - number of medications administered during the hospital stay
- "n_outpatient" - number of outpatient visits in the year before a hospital stay
- "n_inpatient" - number of inpatient visits in the year before the hospital stay
- "n_emergency" - number of visits to the emergency room in the year before the hospital stay
- "medical_specialty" - the specialty of the admitting physician
- "diag_1" - primary diagnosis (Circulatory, Respiratory, Digestive, etc.)
- "diag_2" - secondary diagnosis
- "diag_3" - additional secondary diagnosis
- "glucose_test" - whether the glucose serum came out as high (> 200), normal, or not performed
- "A1Ctest" - whether the A1C level of the patient came out as high (> 7%), normal, or not performed
- "change" - whether there was a change in the diabetes medication ('yes' or 'no')
- "diabetes_med" - whether a diabetes medication was prescribed ('yes' or 'no')
- "readmitted" - if the patient was readmitted at the hospital ('yes' or 'no') 

***Acknowledgments**: Beata Strack, Jonathan P. DeShazo, Chris Gennings, Juan L. Olmo, Sebastian Ventura, Krzysztof J. Cios, and John N. Clore, "Impact of HbA1c Measurement on Hospital Readmission Rates: Analysis of 70,000 Clinical Database Patient Records," BioMed Research International, vol. 2014, Article ID 781670, 11 pages, 2014.*

In [2]:
import pandas as pd
df = pd.read_csv("~/workspace/reducing-hospital-readmissions/data/hospital_readmissions.csv")
df.head()

age  time_in_hospital  n_lab_procedures  ...  change  diabetes_med  readmitted
0  [70-80)                 8                72  ...      no           yes          no
1  [70-80)                 3                34  ...      no           yes          no
2  [50-60)                 5                45  ...     yes           yes         yes
3  [70-80)                 2                36  ...     yes           yes         yes
4  [60-70)                 1                42  ...      no           yes          no

[5 rows x 17 columns]

## 💪 Competition challenge
Create a report that covers the following:

1. What is the most common primary diagnosis by age group? 
2. Some doctors believe diabetes might play a central role in readmission. Explore the effect of a diabetes diagnosis on readmission rates. 
3. On what groups of patients should the hospital focus their follow-up efforts to better monitor patients with a high probability of readmission?

In [4]:
# 1. What is the most common primary diagnosis by age group? 

df_group_diagnosis_by_age = df.groupby(
                                ["age", "diag_1"],
                                sort=False
                            ).size(
                            ).reset_index(
                            name='counts_diag_1_per_age'
)

df_group_diagnosis_by_age.head(5)

age       diag_1  counts_diag_1_per_age
0  [70-80)  Circulatory                   2392
1  [70-80)        Other                   1693
2  [50-60)  Circulatory                   1256
3  [60-70)        Other                   1402
4  [40-50)        Other                    750

In [5]:
df_group_diagnosis_by_age_first = df_group_diagnosis_by_age [ 
    df_group_diagnosis_by_age.groupby(["age", "diag_1"])['counts_diag_1_per_age'].rank() == 1]

df_group_diagnosis_by_age_first.head()

age       diag_1  counts_diag_1_per_age
0  [70-80)  Circulatory                   2392
1  [70-80)        Other                   1693
2  [50-60)  Circulatory                   1256
3  [60-70)        Other                   1402
4  [40-50)        Other                    750

In [6]:
# 2. Some doctors believe diabetes might play a central role in readmission. Explore the effect of a diabetes diagnosis on readmission rates. 

def flag_diabetes_rates (diag_1, readmitted):   
    return 'True' if diag_1 == 'Diabetes' and readmitted == 'yes' else 'False'

df_diabetes = df 

df_diabetes['diabetes_rates'] = df_diabetes.apply( lambda x: flag_diabetes_rates( x.diag_1, x.readmitted), axis = 1)

df_diabetes.head(5)

age  time_in_hospital  ...  readmitted  diabetes_rates
0  [70-80)                 8  ...          no           False
1  [70-80)                 3  ...          no           False
2  [50-60)                 5  ...         yes           False
3  [70-80)                 2  ...         yes           False
4  [60-70)                 1  ...          no           False

[5 rows x 18 columns]

In [7]:
def get_diabetes_rates_porcentage(df):
    
    count_true = 0

    count_false = 0

    for idx, row in df.iterrows():
        if row['diabetes_rates'] == 'True':
            count_true = count_true + 1
        else:
            count_false = count_false + 1
            
    count_all = count_true + count_false
    
    return print((count_true/count_all) * 100)
                
get_diabetes_rates_porcentage(df_diabetes)

3.7479999999999998


In [13]:
# 3. On what groups of patients should the hospital focus their follow-up efforts to better monitor patients with a high probability of readmission?

def get_follow_up_group(df):

    my_column = []
    
    risk_group = ['Missing',
                  'InternalMedicine',
                  'Emergency/Trauma',
                  'Cardiology',
                  'Family/GeneralPractice',
                  'Surgery',
                  'Other']

    for idx, row in df.iterrows(): 

        if not row['readmitted'] == 'yes':
            my_column.append('False')
            continue
        if row['diabetes_med'] == 'yes': 
             my_column.append('True')
        elif row["medical_specialty"] == (risk_group):
            my_column.append('True')
        else:
            my_column.append('False')
    df['follow_up'] = my_column
    
    df_follow_up = df[(df.follow_up == "True")]
    
    return df_follow_up.head(5)


In [14]:
get_follow_up_group(df)

age  time_in_hospital  ...  diabetes_rates  follow_up
2   [50-60)                 5  ...           False       True
3   [70-80)                 2  ...           False       True
7   [60-70)                 1  ...           False       True
15  [80-90)                 8  ...           False       True
16  [70-80)                 3  ...           False       True

[5 rows x 19 columns]